In [1]:
import numpy as np
import collections
import os
from collections import defaultdict,Counter
import tensorflow
from tensorflow.keras.utils import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
import hashlib

In [2]:
b = [0,1,2,3,4,5,6,7,8]
#调用to_categorical将b按照9个类别来进行转换
b = to_categorical(b, 9)
b

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

### Generate Data
#### 小规模

- 基本参数

In [77]:
m = 10
spacesize = 30
n = 15
k = 4
sparse = 0.2

- 生成hash值

In [4]:
hash_list = np.array([np.random.choice(n, spacesize, replace=True) for i in range(m)])
hash_list

array([[ 3, 17, 27, 23,  3, 12,  2, 14, 15, 21, 18,  8, 16, 11, 15],
       [24,  6, 28, 28, 21,  5, 12, 20, 29, 26, 25, 26,  4, 16, 19],
       [ 5, 26, 20,  1,  6, 17, 24, 22,  5,  9,  3, 15,  0, 27, 23],
       [11, 18, 23, 11, 12, 28, 23, 24,  8, 18, 18, 15, 27, 18,  4],
       [24,  2, 11, 17, 18, 22, 11, 21, 23, 25,  1, 28, 28, 11, 15],
       [29, 27, 22, 20, 18,  4,  0, 29, 25, 14, 20, 18, 16,  8, 22],
       [16,  7, 23,  8,  3, 18,  8, 28, 13, 23,  7, 29,  5, 17,  4],
       [ 7,  0, 21, 23, 17, 16, 24,  4, 22,  0, 27, 18, 17, 12,  5],
       [ 7, 14, 13, 24, 26,  5,  2,  3, 16,  9,  5,  3, 21,  8,  1],
       [18, 28, 18,  9, 15,  9, 21, 10,  8, 13, 12, 15, 15,  6,  2]])

#### 真实权重(DNN)
- 基本参数

In [4]:
m = 1000
n = 20
spacesize = 15
k = 3
sparse = 0.2

- 模型训练

In [2]:
def get_subdataset(train_num,test_num,random=True):
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    if random:
        train_indices = np.random.choice(len(train_images), train_num, replace=False)
        test_indices = np.random.choice(len(test_images), test_num, replace=False)
        x_train = train_images[train_indices].astype('float32')
        y_train = train_labels[train_indices]
        x_test = test_images[test_indices].astype('float32')
        y_test = test_labels[test_indices] 
    else:
        x_train = train_images[:train_num].astype('float32')
        y_train = train_labels[:train_num]
        x_test = test_images[:test_num].astype('float32')
        y_test = test_labels[:test_num]    
    return x_train,y_train,x_test,y_test

In [5]:
clients = n
train_num,test_num = 10000,2000
hidden = 1024
clients_models = []
#clients_weights = []
#clients_bias = []
#clients_hash = []
for i in range(clients):
    model = Sequential([ 
          Flatten(input_shape=(28, 28)), 
          Dense(hidden,'relu'), 
          Dense(10, "softmax"),
          ])
    model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
    x_train,y_train,x_test,y_test = get_subdataset(train_num,test_num,random=True)
    model.fit(x=x_train,y=y_train, epochs=30, verbose = 0)
    print("Client ",i+1,end=' ')
    model.evaluate(x_test, y_test)
    clients_models.append(model)



63/63 [==============================] - 0s 681us/step - loss: 4.5674 - accuracy: 0.9520


In [8]:
weights_mat = []
path = "../paras/dnn_weights_10240.npy"
for i,model in enumerate(clients_models):
    hidden_weights = model.get_weights()[2]
    hidden_weights_flat = hidden_weights.reshape(-1)
    weights_mat.append(hidden_weights_flat)
weights_mat = np.array(weights_mat).T
weights_mat = weights_mat
np.save(path,weights_mat)

In [28]:
hidden = 128
model = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(hidden,'relu'),
    Dense(10,'softmax'),
    ])
model.load_weights("test_weights.h5")
hidden_weights = model.get_weights()[0]
#print(hidden_weights.shape)
hidden_weights_flat = hidden_weights.reshape(-1)
#print(hidden_weights_flat.shape)

In [ ]:
weights_sample = hidden_weights[0]
print(weights_sample[:100])
head_set = []
for w in weights_sample:
    we = '{:.5e}'.format(w)
    head_set.append(int(we[-1:]))
head = min(head,min(head_set))
weights_cut = [round(w*(10**head)) for w in weights_sample]
print(weights_cut[:100])

- 生成hash值

In [70]:
def generate_hashlist(weights_mat:np.array):
    # get the most common power
    power_dict = dict()
    for weights in weights_mat:
        for w in weights:
            we = '{:.5e}'.format(w)
            power = we[-1:] #power of para in AeB format(B)
            power_dict[power] = power_dict[power]+1 if power_dict.get(power) else 1
    commond_power = int(max(power_dict.items(),key=lambda x:x[1])[0])
    #commond_power = int(max(power_dict.keys()))
    # compute the hash value of head and its value space
    hash_list = []
    max_head, min_head = -float("inf"),float("inf")
    for i,weights in enumerate(weights_mat):
        weights_head = [round(w*(10**commond_power)) for w in weights]
        max_head,min_head = max(max_head,np.max(weights_head)), min(min_head,np.min(weights_head))
        hash_list.append([hash(str(wh)) for wh in weights_head])
    hash_size = max_head - min_head + 1
    return hash_list,hash_size

- 统计不重复元素的频数  $O(n)$ 

In [14]:
def get_stat(h):
    stat_dict = defaultdict(list)
    for i,s in enumerate(h):
        stat_dict[s].append(i)
    stat_keys = list(stat_dict.keys())
    #stat = [(key,len(stat_dict[key])) for key in stat_keys]
    return stat_dict

In [75]:
para_size = m
path = "./weights/dnn_weights.npy"
weights_mat = np.load(path)
weights_mat = weights_mat[:para_size]
hash_list, hash_size = generate_hashlist(weights_mat)
stat_dict = [get_stat(h) for h in hash_list]
print(len(hash_list),len(hash_list[0]),hash_size)
#stat_dict

1000 20 58


### TopK

- 提取频数topk的client

In [61]:
def BucketFrequent(k,spacesize,stat):
    Bucket = [[] for i in range(spacesize)]
    for s in stat:
        Bucket[s[1]].append(s)
    r = 0
    flag = False
    result = []
    for i in range(len(Bucket)-1,-1,-1):
        if not flag and Bucket[i] != []:
            for s in Bucket[i]:
                result.append(s)
                r += 1
                if r == k:
                    flag = True
                    break
    return result

In [32]:
#取父/子节点基本操作
def lchild(node):
    return node << 1
def rchild(node):
    return node << 1 | 1
def father(node):
    return node >> 1
#上浮节点，用于向堆插入新节点
def heap_up(heap,node):
    val = heap[node]
    while father(node)>0 and val[1]<heap[father(node)][1]:
        heap[node] = heap[father(node)]
        node = father(node)
    heap[node] = val
#下沉节点，用于调整堆
def heap_down(heap,node,k):
    root = node #root作为变量逐步下沉
    val = heap[node] #存储node原值
    while lchild(root) <= k:
        child = lchild(root) #先选取左子节点
        if rchild(root) <= k and child < rchild(root): #如果右更小，选取右子节点
            child = rchild(root)
        #验证确实当前节点值大于选取的子节点，则交换
        if heap[child][1] < val[1]:
            heap[root] = heap[child]
            root = child
        else: #否则则找到位置，结束循环
            break
    heap[root] = val #最后赋值
#堆排序输出
def heap_sort(heap):
    for i in range(len(heap)-1, 0, -1):
        heap[1], heap[i] = heap[i], heap[1]
        heap_down(heap, 1, i)
#主函数
def HeapFrequent(k,stat):
    #stat = collections.Counter(nums)
    #stat = list(stat.items())
    heap = [(0,0)] #占位上界
    #使用heap_up()建堆 规模：k+1(包括占位上界) 
    for i in range(k):
        heap.append(stat[i])
        heap_up(heap, len(heap)-1)
    #使用heap_down()维护堆 新元素大于堆顶即下沉
    for i in range(k,len(stat)):
        if stat[i][1] > heap[1][1]: #heap[1]为堆顶
            heap[1] = stat[i] #去除原根（堆顶）节点
            heap_down(heap,1,k) #将node下沉
    #使用heap_sort()排序，并倒序得到从大到小结果
    heap_sort(heap)
    result = heap[1:][::-1]
    return result

In [29]:
def BucketElement(stat_dict,k,spacesize):
    stat_keys = list(stat_dict.keys())
    stat = [(key,len(stat_dict[key])) for key in stat_keys]
    #frequent = BucketFrequent(k,spacesize,stat)
    frequent = HeapFrequent(k,stat)
    #print(frequent)
    elements = [(freq[0],freq[1],stat_dict[freq[0]]) for freq in frequent]
    #print(elements)
    return frequent,elements

In [80]:
def one_hot(x,n):
    one_hot = np.zeros(n)
    for i in x:
        one_hot[i] = 1
    return one_hot

In [ ]:
m = para_size
n = clients

In [87]:
Sp = []
Lp = []
print("Top",k,"clients in each para")
for std in stat_dict:
    frequents,elements = BucketElement(std,k,spacesize)
    #print("F",frequents,elements)
    select = []
    l = 0
    for i in range(len(elements)):
        select += elements[i][2]
        l += elements[i][1]
    select = one_hot(select, n)
    #print(select)
    Sp.append(select)
    Lp.append(l)
Sp = np.mat(Sp)
# 根据大小排序
Lpindex = sorted(range(len(Lp)), key=lambda k: Lp[k], reverse=True)
Sp = Sp[Lpindex]
# 求交集
S_inter = np.ones((1,n))
B_inter = []
B_sparse = []
for i,s in enumerate(Sp):
    s_mat = np.diag(s.tolist()[0])
    flag = np.matmul(S_inter,s.T)
    if flag > 0:
        S_inter = np.matmul(S_inter,s_mat)
        B_inter.append(i)
    else:
        B_sparse.append(i)
print("S_inter",S_inter)
print("Inter Client:",np.argmax(S_inter),"contains paras",len(B_inter))
print("Sparse paras:",len(B_sparse))
print(B_sparse)
print("Sparse percent",len(B_sparse) / m)

Top 3 clients in each para
S_inter [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Inter Client: 1 contains paras 787
Sparse paras: 213
[6, 7, 8, 12, 13, 14, 20, 21, 30, 31, 32, 34, 37, 43, 44, 51, 52, 60, 66, 68, 70, 71, 74, 75, 79, 84, 85, 90, 91, 94, 95, 100, 102, 104, 106, 113, 116, 118, 123, 126, 128, 129, 133, 136, 141, 146, 157, 162, 167, 174, 175, 180, 184, 186, 187, 193, 195, 197, 198, 202, 203, 207, 208, 226, 227, 228, 229, 232, 236, 243, 245, 246, 250, 257, 262, 264, 271, 274, 275, 276, 277, 280, 282, 289, 290, 293, 294, 299, 302, 306, 309, 311, 313, 315, 320, 322, 327, 331, 332, 334, 338, 339, 340, 342, 345, 346, 348, 349, 356, 357, 365, 366, 373, 375, 384, 397, 398, 400, 401, 407, 408, 416, 425, 440, 441, 454, 456, 458, 460, 462, 464, 467, 470, 471, 480, 486, 489, 490, 501, 505, 511, 512, 513, 520, 529, 530, 532, 533, 534, 542, 547, 550, 553, 554, 556, 562, 567, 572, 573, 577, 578, 580, 582, 587, 588, 592, 602, 605, 606, 613, 621, 626, 627, 630, 633, 635, 6